In [78]:
import pandas as pd
import numpy as np
#import geopandas as gpd
import urllib#pour récupérer les données
import bs4#pour rendre lisibles les données
import re

Récupération des députés sur le site de l'Assemblée Nationale

In [2]:
def Scrap (url):
    req = urllib.request.Request(url)
    html = urllib.request.urlopen(req).read()
    page = bs4.BeautifulSoup(html, "lxml")
    return page

In [3]:
def ToDf (table) :
    Listgrossiere=pd.read_html(str(table))#on transforme le tableau en liste de dataframes
    listeDf=Listgrossiere[0]#on récupère le dataframe
    return listeDf

In [4]:
page=Scrap('http://www2.assemblee-nationale.fr/sycomore/resultats/(offset)/'+str(0)+'/(query)/IiBTRUxFQ1QgbV9jb2RlX2RlcHV0ZSwgaWRfYWN0ZXVyX3RyaWJ1biwgbGVnX21heF90cmlidW4sIG5vbSwgbm9tX2FmZmljaGUsIHByZW5vbSwgZGF0ZV9uYWlzLCBkYXRlX2RlY2VzIEZST00gZGVwdXRlIFdIRVJFIDE9MSAgQU5EIG1fY29kZV9kZXB1dGUgSU4gKFNFTEVDVCBtX2NvZGVfZGVwdXRlIEZST00gbWFuZGF0LCBkZXBhcnRlbWVudCAgV0hFUkUgbWFuZGF0Lm1fbnVtX2RlcCA9IGRlcGFydGVtZW50Lm1fbnVtX2RlcCBBTkQgbWFuZGF0Lm1fdHlwZV9tYW5kYXQgPSAxICkgT1JERVIgQlkgZGF0ZV9uYWlzIERFU0Mi')
table = page.find('table')
liste=ToDf(table)
for i in range(32):
    page=Scrap('http://www2.assemblee-nationale.fr/sycomore/resultats/(offset)/'+str((i+1)*500)+'/(query)/IiBTRUxFQ1QgbV9jb2RlX2RlcHV0ZSwgaWRfYWN0ZXVyX3RyaWJ1biwgbGVnX21heF90cmlidW4sIG5vbSwgbm9tX2FmZmljaGUsIHByZW5vbSwgZGF0ZV9uYWlzLCBkYXRlX2RlY2VzIEZST00gZGVwdXRlIFdIRVJFIDE9MSAgQU5EIG1fY29kZV9kZXB1dGUgSU4gKFNFTEVDVCBtX2NvZGVfZGVwdXRlIEZST00gbWFuZGF0LCBkZXBhcnRlbWVudCAgV0hFUkUgbWFuZGF0Lm1fbnVtX2RlcCA9IGRlcGFydGVtZW50Lm1fbnVtX2RlcCBBTkQgbWFuZGF0Lm1fdHlwZV9tYW5kYXQgPSAxICkgT1JERVIgQlkgZGF0ZV9uYWlzIERFU0Mi')
    table = page.find('table')#on récupère le seul tableau de la page qui liste des députés
    listeDf=ToDf(table)
    liste=pd.concat([liste,listeDf])#on récupère le dataframe et on l'ajoute à la liste
liste=liste.reset_index()#on recrée un index afin de ne pas s'arrêter à 500 puis recommencer comme dans les pages web
listeDepute=liste[['Nom','Né le']]#on récupère les variables qui nous intéressent  

On s'intéresse ensuite à la récupération des mandats de chaque député (partie en cours).

In [ ]:
page=Scrap('http://www2.assemblee-nationale.fr/sycomore/fiche/(num_dept)/'+str(5))
listeMandat=page
for i in range(len(listeDepute)-1) :
    page=Scrap('http://www2.assemblee-nationale.fr/sycomore/fiche/(num_dept)/'+str(i+7))
    

In [93]:
page=Scrap('http://www2.assemblee-nationale.fr/sycomore/fiche/(num_dept)/'+str(5))
depute=page.findAll('dl', {'class' : "deputes-liste-attributs sycomore"})

In [111]:
for mandat in depute :
    columns=mandat.findAll('dt')
    columns=re.sub('<[^<]+?>', '', str(columns))  
    lines=mandat.findAll('dd')
    lines=re.sub('<[^<]+?>', '', str(lines))
    lines=re.sub('\t', '', str(lines))
    lines=re.sub("\n", '', str(lines))
    line=''
    for i in str(lines) :
        line+=i
    line=str(line)
    df=pd.DataFrame([columns, line])    
    print(df)

                                                   0
0  [Régime politique, Législature, Mandat, Départ...
1  [Cinquième République - Assemblée nationale, V...
                                                   0
0  [Régime politique, Législature, Mandat, Départ...
1  [Cinquième République - Assemblée nationale, V...
                                                   0
0  [Régime politique, Législature, Mandat, Départ...
1  [Cinquième République - Assemblée nationale, V...
                                                   0
0                               [Sénateur, Sénateur]
1  [du 3 octobre 1983 au  1er octobre 1992, du 2 ...
